# Adversarial Robustness Toolkit

* Documentación: https://adversarial-robustness-toolbox.readthedocs.io/en/latest/
* Código: https://github.com/Trusted-AI/adversarial-robustness-toolbox
* Ejemplos: https://github.com/Trusted-AI/adversarial-robustness-toolbox/tree/main/examples

## Instalación

In [1]:
!pip install adversarial-robustness-toolbox==1.8.1

     |████████████████████████████████| 1.1 MB 5.3 MB/s 
     |████████████████████████████████| 3.3 MB 30.3 MB/s 
     |████████████████████████████████| 26.3 MB 1.9 MB/s 
  Attempting uninstall: llvmlite
    Found existing installation: llvmlite 0.34.0
    Uninstalling llvmlite-0.34.0:
      Successfully uninstalled llvmlite-0.34.0
  Attempting uninstall: numba
    Found existing installation: numba 0.51.2
    Uninstalling numba-0.51.2:
      Successfully uninstalled numba-0.51.2


In [2]:
import keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
import numpy as np
import matplotlib.pyplot as plt
from art.utils import load_mnist

In [3]:
from tensorflow.python.framework.ops import disable_eager_execution
disable_eager_execution()

import warnings
warnings.filterwarnings('ignore')

In [4]:
%matplotlib inline

## Cargar datos

In [5]:
(x_train, y_train), (x_test, y_test), min_pixel_value, max_pixel_value = load_mnist()

## Entrenar modelo

In [6]:
from art.estimators.classification import KerasClassifier

In [7]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(10, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

victim = KerasClassifier(model=model, clip_values=(min_pixel_value, max_pixel_value), use_logits=False)

victim.fit(x_train, y_train, batch_size=128, nb_epochs=5)

Train on 60000 samples
Epoch 1/5
60000/60000 [==============================] - 17s 284us/sample - loss: 0.2147 - accuracy: 0.9353
Epoch 2/5
60000/60000 [==============================] - 4s 71us/sample - loss: 0.0648 - accuracy: 0.9791
Epoch 3/5
60000/60000 [==============================] - 4s 71us/sample - loss: 0.0452 - accuracy: 0.9860
Epoch 4/5
60000/60000 [==============================] - 4s 70us/sample - loss: 0.0364 - accuracy: 0.9887
Epoch 5/5
60000/60000 [==============================] - 4s 70us/sample - loss: 0.0282 - accuracy: 0.9911


In [8]:
predictions_test = victim.predict(x_test)
accuracy = np.sum(np.argmax(predictions_test, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print("Accuracy on test examples: {:.2f}%".format(accuracy * 100))

Accuracy on test examples: 99.14%


## Ataque de extracción

In [9]:
# Más ataques en
# https://adversarial-robustness-toolbox.readthedocs.io/en/latest/modules/attacks/extraction.html

from art.attacks.extraction import CopycatCNN

In [10]:
max_requests = 5000 
shuffle = np.random.permutation(len(x_test))
x_stolen = x_test[shuffle[:max_requests]]
y_stolen = y_test[shuffle[:max_requests]]

In [11]:
model_stolen = Sequential()
model_stolen.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)))
model_stolen.add(MaxPooling2D(pool_size=(2, 2)))
model_stolen.add(Conv2D(64, (3, 3), activation='relu'))
model_stolen.add(MaxPooling2D(pool_size=(2, 2)))
model_stolen.add(Dropout(0.25))
model_stolen.add(Flatten())
model_stolen.add(Dense(128, activation='relu'))
model_stolen.add(Dense(10, activation='softmax'))

model_stolen.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [12]:
attack = CopycatCNN(classifier=victim, nb_epochs=5, nb_stolen=max_requests, use_probability=True)
classifier_stolen = KerasClassifier(model_stolen, clip_values=(0, 1), use_logits=False)
classifier_stolen = attack.extract(x_stolen, y_stolen, thieved_classifier=classifier_stolen)

Train on 5000 samples
Epoch 1/5
5000/5000 [==============================] - 25s 5ms/sample - loss: 0.2969 - accuracy: 0.9088
Epoch 2/5
5000/5000 [==============================] - 25s 5ms/sample - loss: 0.1248 - accuracy: 0.9654
Epoch 3/5
5000/5000 [==============================] - 25s 5ms/sample - loss: 0.0975 - accuracy: 0.9762
Epoch 4/5
5000/5000 [==============================] - 25s 5ms/sample - loss: 0.0835 - accuracy: 0.9810
Epoch 5/5
5000/5000 [==============================] - 25s 5ms/sample - loss: 0.0792 - accuracy: 0.9806


In [13]:
predictions_stolen = classifier_stolen.predict(x_test)
accuracy = np.sum(np.argmax(predictions_stolen, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print("Accuracy on test examples (stolen model): {:.2f}%".format(accuracy * 100))

Accuracy on test examples (stolen model): 98.58%


# Defensas

In [14]:
# Más defensas en
# https://adversarial-robustness-toolbox.readthedocs.io/en/latest/modules/defences/postprocessor.html

from art.defences.postprocessor import Rounded, GaussianNoise, ReverseSigmoid

In [16]:
postprocessor_rounded = Rounded(decimals=1)

In [17]:
postprocessor_gaussian = GaussianNoise(scale=0.1)

In [18]:
postprocessor_reverse_sigmoid = ReverseSigmoid(beta=1.0, gamma=0.5)

In [19]:
victim_defense = KerasClassifier(model=model, clip_values=(min_pixel_value, max_pixel_value), use_logits=False, postprocessing_defences=postprocessor_reverse_sigmoid)
victim_defense.fit(x_train, y_train, batch_size=128, nb_epochs=5)

Train on 60000 samples
Epoch 1/5
60000/60000 [==============================] - 4s 73us/sample - loss: 0.0257 - accuracy: 0.9918
Epoch 2/5
60000/60000 [==============================] - 4s 70us/sample - loss: 0.0218 - accuracy: 0.9928
Epoch 3/5
60000/60000 [==============================] - 4s 70us/sample - loss: 0.0174 - accuracy: 0.9942
Epoch 4/5
60000/60000 [==============================] - 4s 71us/sample - loss: 0.0151 - accuracy: 0.9948
Epoch 5/5
60000/60000 [==============================] - 4s 71us/sample - loss: 0.0141 - accuracy: 0.9954


In [20]:
predictions_victim_defense = victim_defense.predict(x_test)
accuracy = np.sum(np.argmax(predictions_victim_defense, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print("Accuracy on test examples (protected): {:.2f}%".format(accuracy * 100))

Accuracy on test examples (protected): 99.19%


In [21]:
model_stolen_protected = Sequential()
model_stolen_protected.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)))
model_stolen_protected.add(MaxPooling2D(pool_size=(2, 2)))
model_stolen_protected.add(Conv2D(64, (3, 3), activation='relu'))
model_stolen_protected.add(MaxPooling2D(pool_size=(2, 2)))
model_stolen_protected.add(Dropout(0.25))
model_stolen_protected.add(Flatten())
model_stolen_protected.add(Dense(128, activation='relu'))
model_stolen_protected.add(Dense(10, activation='softmax'))
model_stolen_protected.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [22]:
attack_protected = CopycatCNN(classifier=victim_defense, nb_epochs=5, nb_stolen=max_requests, use_probability=True)
classifier_stolen_protected = KerasClassifier(model_stolen_protected, clip_values=(0, 1), use_logits=False)
classifier_stolen_protected = attack_protected.extract(x_stolen, y_stolen, thieved_classifier=classifier_stolen_protected)

Train on 5000 samples
Epoch 1/5
5000/5000 [==============================] - 25s 5ms/sample - loss: 2.3026 - accuracy: 0.1034
Epoch 2/5
5000/5000 [==============================] - 24s 5ms/sample - loss: 2.3026 - accuracy: 0.1018
Epoch 3/5
5000/5000 [==============================] - 24s 5ms/sample - loss: 2.3026 - accuracy: 0.1052
Epoch 4/5
5000/5000 [==============================] - 24s 5ms/sample - loss: 2.3026 - accuracy: 0.1002
Epoch 5/5
5000/5000 [==============================] - 24s 5ms/sample - loss: 2.3026 - accuracy: 0.0920


In [23]:
predictions_stolen_protected = classifier_stolen_protected.predict(x_test)
accuracy = np.sum(np.argmax(predictions_stolen_protected, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print("Accuracy on test examples against protected model (stolen model): {:.2f}%".format(accuracy * 100))

Accuracy on test examples against protected model (stolen model): 9.74%
